<a href="https://colab.research.google.com/github/sukritichawla/legaldocs_ST/blob/main/notebooks/01_simplification_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# 1. GPU check
!nvidia-smi

# 2. Install dependencies
!pip -q install -U transformers datasets sentencepiece accelerate huggingface_hub python-dotenv feedparser beautifulsoup4 sacrebleu rouge-score gradio



# 3. Clone or pull repo
from getpass import getpass
token = getpass("Enter your GitHub PAT: ")
USERNAME = "sukritichawla"
REPO = "legaldocs_ST"
REPO_URL = f"https://{token}@github.com/{USERNAME}/{REPO}.git"
!git clone $REPO_URL


# 4. Go inside repo
%cd /content/legaldocs_ST

# 5. Hugging Face login
from huggingface_hub import login
login()


Thu Sep  4 05:48:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P0             29W /   70W |    2750MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [27]:
from transformers import pipeline

# Load a summarization model
simplifier = pipeline("summarization", model="facebook/bart-large-cnn")

# Example legal text
text = """Whereas it is expedient to provide for the protection of individuals in respect of their personal data, and to establish mechanisms for the regulation of data processing by public and private entities, it is hereby enacted that no person shall collect, store, or disclose any personal information of another except in accordance with the provisions of this Act and the rules made thereunder"""

# Run simplification
result = simplifier(text, max_length=80, min_length=30, do_sample=False)
print("Simplified English:", result[0]["summary_text"])


Device set to use cuda:0
Your max_length is set to 80, but your input_length is only 71. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=35)


Simplified English: It is hereby enacted that no person shall collect, store, or disclose any personal information of another except in accordance with the provisions of this Act and the rules made thereunder.


In [2]:
simplifier_t5 = pipeline("summarization", model="google/flan-t5-base")

result_t5 = simplifier_t5(text, max_length=80, min_length=30, do_sample=False)
print("Simplified English (FLAN-T5):", result_t5[0]["summary_text"])


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Your max_length is set to 80, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Simplified English (FLAN-T5): In the case of loans, it is expedient to provide for the extension of loans to students pursuing higher education under certain conditions. Moreover, if a loan is extended to a student pursuing a higher education degree under certain circumstances, the student must be able to repay the loan in full.


In [9]:
!pwd

/content/legaldocs_ST


In [10]:
import pandas as pd

examples = [text]
outputs = [result[0]["summary_text"]]

df = pd.DataFrame({"original": examples, "simplified": outputs})
df.to_csv("data/processed/simplified_samples.csv", index=False)
print("Saved to data/processed/simplified_samples.csv")


Saved to data/processed/simplified_samples.csv


In [20]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	legaldocs_ST/

nothing added to commit but untracked files present (use "git add" to track)


In [21]:
!git ls-files --others --exclude-standard


legaldocs_ST/


In [24]:
from getpass import getpass
token = getpass("Enter your GitHub PAT again: ")

USERNAME = "sukritichawla"
REPO = "legaldocs_ST"

!git remote set-url origin https://{token}@github.com/{USERNAME}/{REPO}.git


Enter your GitHub PAT again: ··········


In [25]:
!git push origin main


To https://github.com/sukritichawla/legaldocs_ST.git
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/sukritichawla/legaldocs_ST.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [26]:
!git pull origin main

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 10.52 KiB | 3.51 MiB/s, done.
From https://github.com/sukritichawla/legaldocs_ST
 * branch            main       -> FETCH_HEAD
   ff88df6..c0ffc45  main       -> origin/main
hint: You have divergent branches and need to specify how to reconcile them.
hint: You can do so by running one of the following commands sometime before
hint: your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default

In [28]:
from transformers import pipeline

simplifier = pipeline("text2text-generation", model="google/flan-t5-base")

text = "Whereas it is expedient to provide for the protection of individuals in respect of their personal data, and to establish mechanisms for the regulation of data processing by public and private entities, it is hereby enacted that no person shall collect, store, or disclose any personal information of another except in accordance with the provisions of this Act and the rules made thereunder"
output = simplifier("Simplify: " + text, max_length=100)[0]["generated_text"]

print("Simplified:", output)


Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Simplified: This Act shall provide for the protection of individuals in respect of their personal data, and to establish mechanisms for the regulation of data processing by public and private entities, and it shall be hereby enacted that no person shall collect, store, or disclose any personal information of another except in accordance with the provisions of this Act and the rules made thereunder.


In [29]:
from transformers import pipeline

simplifier = pipeline("text2text-generation", model="google/flan-t5-base")

text = "This Act shall provide for the protection of individuals in respect of their personal data, and to establish mechanisms for the regulation of data processing by public and private entities, and it shall be hereby enacted that no person shall collect, store, or disclose any personal information of another except in accordance with the provisions of this Act and the rules made thereunder."
output = simplifier("Simplify: " + text, max_length=100)[0]["generated_text"]

print("Simplified:", output)

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Simplified: This Act shall provide for the protection of individuals in respect of their personal data, and to establish mechanisms for the regulation of data processing by public and private entities, and it shall be hereby enacted that no person shall collect, store, or disclose any personal information of another except in accordance with the provisions of this Act and the rules made thereunder.


In [35]:
from transformers import pipeline

# Step 1: Summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Step 2: Simplifier (use PEGASUS paraphrase)
simplifier = pipeline("text2text-generation", model="tuner007/pegasus_paraphrase")

text = """This Act shall provide for the protection of individuals in respect of their personal data,
and to establish mechanisms for the regulation of data processing by public and private entities,
and it shall be hereby enacted that no person shall collect, store, or disclose any personal
information of another except in accordance with the provisions of this Act and the rules made thereunder."""

# Step 1: Summarization
summary = summarizer(text, max_length=80, min_length=30, do_sample=False)[0]['summary_text']

# Step 2: Simplification
simplified = simplifier("Paraphrase in simple English: " + summary, max_length=100, num_return_sequences=1)[0]['generated_text']

print("---- Original ----")
print(text, "\n")
print("---- Step 1: Summary (BART) ----")
print(summary, "\n")
print("---- Step 2: Simplified (PEGASUS) ----")
print(simplified)


Device set to use cuda:0


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cuda:0
Your max_length is set to 80, but your input_length is only 74. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


---- Original ----
This Act shall provide for the protection of individuals in respect of their personal data,
and to establish mechanisms for the regulation of data processing by public and private entities,
and it shall be hereby enacted that no person shall collect, store, or disclose any personal
information of another except in accordance with the provisions of this Act and the rules made thereunder. 

---- Step 1: Summary (BART) ----
This Act shall provide for the protection of individuals in respect of their personal data. It shall be hereby enacted that no person shall collect, store, or disclose any personal information of another except in accordance with the provisions of this Act. 

---- Step 2: Simplified (PEGASUS) ----
This Act will provide for the protection of individuals in respect of their personal data.
